In [ ]:
import galsim
from om10 import DB
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

import sys
realizer_path = os.path.join(os.environ['SLREALIZERDIR'], 'slrealizer')
sys.path.insert(0, realizer_path)
from realize_om10 import OM10Realizer
from utils.utils import *
from utils.constants import *
#from realize_sl import SLRealizer

%matplotlib inline
%load_ext autoreload
%autoreload 2

# Includes:
# 1. Determining nx, ny, and pixel scale for GalSim's drawImage and 
# 2. Comparing the true and emulated images
# 3. Comparing the HSM output with the analytically derived properties

## 1. Checking that all functions of OM10Realizer run as expected

I write this test in a notebook because I'll need to look at images for each (nx, ny, pixel scale) configuration and qualitatively judge whether it will be okay for HSM shape estimation. First, I define the filepath for the test lens catalog and test observation history. I created these files just for testing purposes; they only have one row each.

In [ ]:
data_path = os.path.join(os.environ['SLREALIZERDIR'], 'data')

test_catalog_f = os.path.join(data_path, 'test_catalog.fits')
observation_f = os.path.join(data_path, 'twinkles_observation_history.csv')

In [ ]:
f = os.path.join(data_path, 'qso_mock.fits')
from astropy.io import fits
hdul = fits.open(f)

In [ ]:
d = hdul[1].data


In [ ]:
from astropy.table import Table
t = Table(d)
t

We will read them in as OM10 DB and Pandas dataframe, respectively, and use them to create our OM10Realizer object.

In [ ]:
test_db = DB(catalog=test_catalog_f)
#test_db.select_random(maglim=23.3, area=18000.0, IQ=0.75)
test_db.paint(synthetic=True)

test_obs = pd.read_csv(observation_f).sample(1, random_state=123).reset_index(drop=True)

om10realizer = OM10Realizer(observation=test_obs, catalog=test_db, debug=True)

In [ ]:
from om10 import DB
from realize_om10 import OM10Realizer

# Create OM10 DB instance
om10_db = DB(catalog=catalog_path)
# Simulate colors (ugrizy magnitudes)
om10_db.paint(synthetic=True)

# Read observation data into a dataframe
minion_df = pd.read_csv(minion1016_path)
# Create OM10Realizer instance
realizer = OM10Realizer(observation=minion_df,
                        catalog=om10_db)

# Generate source table 
# and save as output_source_table_path
realizer.make_source_table_vectorized(save_file=output_source_table_path)
# Generate object table from source table at output_source_table_path
# and save as output_object_table_path
realizer.make_object_table(sourceTablePath=output_source_table_path,
                           objectTablePath=output_object_table_path)

These are equivalents of the lensInfo and obsInfo parameters that are input to many of the methods in OM10Realizer. We save the values to test individual methods.

In [ ]:
test_lensInfo = test_db.sample[0]
test_obsInfo = test_obs.loc[0]

In [ ]:
test_obsInfo

In [ ]:
test_lensInfo

In [ ]:
np.radians(test_lensInfo['PHIE']), np.deg2rad(test_lensInfo['PHIE'])

In [ ]:
s = galsim.Shear(e1=0.4, e2=-0.3)
#print("q", np.exp(-s.eta))
#print("beta", s.beta)
l = galsim.Gaussian(sigma=0.6, flux=21.0).shear(s)
#q = galsim.Gaussian(sigma=0.0, flux=11.0).shift((5.0, 7.0))
#psf = galsim.Gaussian(sigma=7.0)
total = l

#total = l
#convolved = galsim.Convolve([total, psf])

convolved = total
img = convolved.drawImage(nx=64, ny=64, method='no_pixel', scale=0.1)


In [ ]:
plt.imshow(img.array)
plt.title("Sample SDSS galaxy (non-lens)")
plt.colorbar()

In [ ]:
psf = galsim.Gaussian(sigma=1.0)
psf_img = psf.drawImage(nx=64, ny=64, scale=0.2, method='no_pixel')
plt.imshow(psf_img.array)
plt.title("Sample PSF")
plt.colorbar()

In [ ]:
q = ((1.0-s.e)/(1+s.e))**0.5
beta = 0.5*np.arctan(0.3/0.7)
q, beta

In [ ]:
a = 13.0/(q**0.5)
b = 13.0*q**0.5
print("a, b:", a, b)
lam1 = a**2.0
lam2 = b**2.0
Sig_11 = lam1*(np.cos(beta))**2.0 + lam2*(np.sin(beta))**2.0
Sig_12 = (lam1 - lam2)*np.cos(beta)*np.sin(beta)
Sig_22 = lam1*(np.sin(beta))**2.0 + lam2*(np.cos(beta))**2.0
print("Sig", Sig_11, Sig_12, Sig_22)
new_e1 = (Sig_11 - Sig_22)/(Sig_11 + Sig_22)
new_e2 = 2.0*Sig_12/(Sig_11 + Sig_22)
print("out e1, e2", new_e1, new_e2)

In [ ]:
img.FindAdaptiveMom().moments_sigma

In [ ]:
lens_flux = 37.0/48.0
q_flux = 11.0/48.0
Q_11 = lens_flux*Sig_11 + 

In [ ]:
0.5*np.arctan(2.0*Sig_12/(Sig_11-Sig_22))
#0.5*(Sig_11+Sig_22+np.sign(Sig_11-Sig_22)*((Sig_11-Sig_22)**2.0 + 4.0*Sig_12**2.0)**0.5)

In [ ]:
img.calculateMomentRadius(rtype='trace')/((Sig_11+Sig_22)*0.5)**0.5, img.calculateMomentRadius(rtype='det')/(Sig_11*Sig_22-Sig_12**2.0)**0.25

In [ ]:
img.calcula

In [ ]:
np.sign(-3)

In [ ]:
plt.imshow(img.array)
plt.colorbar()

In [ ]:
img.calculateMomentRadius(rtype='trace')

In [ ]:
(13.0**2.0 + 7.0**2.0)**0.5

In [ ]:
mom = img.FindAdaptiveMom()

In [ ]:
mom.observed_shape.e1, mom.observed_shape.e2

In [ ]:
mom.moments_sigma

### 1.1 Converting OM10 catalog values into GalSim input parameters

In [ ]:
galsimInput = om10realizer._om10_to_galsim(test_lensInfo, test_obsInfo['filter'])
galsimInput

### 1.2 Drawing the true image via GalSim's drawImage

In [ ]:
trueImg = om10realizer.draw_system(lensInfo=om10_db.sample[0], 
                                   obsInfo=minion_df.loc[0])
plt.imshow(trueImg.array)
plt.colorbar()

### 1.3 Running HSM's shape estimation on the true image

In [ ]:
hsmOutput = om10realizer.estimate_hsm(lensInfo=test_lensInfo, obsInfo=test_obsInfo)
hsmOutput

### 1.4 Drawing the emulated image via GalSim's drawImage

In [ ]:
emulatedImg = om10realizer.draw_emulated_system(lensInfo=om10_db.sample[0], 
                                                obsInfo=minion_df.loc[0])
plt.imshow(emulatedImg.array)
plt.colorbar()

In [ ]:
emulatedImg = om10realizer.draw_emulated_system(lensInfo=test_lensInfo, obsInfo=test_obsInfo)
plt.imshow(emulatedImg.array)
plt.colorbar()

## 2. Comparing the true image vs. emulated image

### 2.1 First moments x, y

Qualitatively, the two images look similar. When we take the difference of the images, it seems that their centers have a slight offset. Let's investigate why.

In [ ]:
plt.imshow(trueImg.array-emulatedImg.array)
plt.colorbar()

In [ ]:
estEmulated = emulatedImg.FindAdaptiveMom()
pixel_to_physical(estEmulated.moments_centroid.x, om10realizer.nx, om10realizer.pixel_scale)

### 2.2 Second moments (half-light radius)

If the shear is not too dramatic, `calculateHLR` works fairly well to estimate the true HLR. Let us first compare the HLR of the emulated image to the true HLR.

In [ ]:
from utils.utils import physical_to_pixel
phys_x = physical_to_pixel(hsmOutput['x'], canvasSize=om10realizer.nx, pixel_scale=om10realizer.pixel_scale)
phys_y = physical_to_pixel(hsmOutput['y'], canvasSize=om10realizer.ny, pixel_scale=om10realizer.pixel_scale)
pixelCenter = galsim.PositionD(x=phys_x, y=phys_y)

In [ ]:
emulated_hlr = emulatedImg.calculateHLR(center=pixelCenter)

The HLRs of the true and emulated images do not agree to floating-point precision.

In [ ]:
print(np.allclose(hsmOutput['hlr'], emulated_hlr))
print(hsmOutput['hlr'], emulated_hlr)

It turns out that the emulated image is 1% bigger!

In [ ]:
print((emulated_hlr-hsmOutput['hlr'])/hsmOutput['hlr'])

We've compared the half-light radius, but what goes in the source table (and stored in `hsmOutput['trace']`) is the trace of the second moment from `calculateMomentRadius(rtype='trace')`

In [ ]:
hsmOutput['trace']

### 2.3 Flux

It doesn't make sense to use HSM's flux estimation (`moments.amp`) when we have access to the true flux, i.e. the sum of the lens flux and the four (quasar) image fluxes is the true total flux. So we bypass HSM and store the sum of the true image's pixel values as flux. So we expect that the flux values of true and emulated images agree within floating-point precision.

In [ ]:
l_flux = galsimInput['flux']
g_fluxes = [galsimInput['flux_' + str(i)] for i in range(4)]
true_tot_flux = l_flux + np.sum(g_fluxes)
print(np.allclose(true_tot_flux, hsmOutput['apFlux']))

## 3. Comparing the HSM output with the analytically derived properties

### 3.1 Second moments

Here, we compare GalSim's numerical calculation of the second moment with the analytically derived moments. This is a good test for the analytical calculations! First, we render a high-res image of our test object so that GalSim can do a more precise moment calculation.

In [ ]:
om10realizer.nx, om10realizer.ny = 101, 101
om10realizer.pixel_scale = 0.05
trueImgHR = om10realizer.draw_system(lensInfo=test_lensInfo, obsInfo=test_obsInfo)
plt.imshow(trueImgHR.array)
plt.colorbar()

We run our analytical moment calculation on the object parameters and the HSM shape estimation on the image.

In [ ]:
derivedProps = om10realizer._om10_to_lsst(lensInfo=test_lensInfo, obsInfo=test_obsInfo)
outputHSM = trueImgHR.FindAdaptiveMom()

The GalSim function `calculateMomentRadius(rtype='trace')` returns $\sqrt{\frac{I_{xx} + I_{yy}}{2}}$ whereas setting the parameter `rtype='det'` returns $\left( I_{xx} \times I_{yy} - I_{xy}^2 \right)^{1/4}$. We make adjustments to get just the trace and the determinant. Alternatively, we can call `FindAdaptiveMom` on the image. We do both here.

In [ ]:
# Numerical trace, determinant
anaIx = derivedProps['x']
anaIy = derivedProps['y']
anaCentroid = galsim.PositionD(x=physical_to_pixel(anaIx, 101, 0.05), y=physical_to_pixel(anaIy, 101, 0.05))

numTrace = trueImgHR.calculateMomentRadius(rtype='trace', center=anaCentroid)**2.0*2.0
numDet = trueImgHR.calculateMomentRadius(rtype='det', center=anaCentroid)**4.0
numDetHSM = (outputHSM.moments_sigma*om10realizer.pixel_scale)**4.0
print("Numerical trace: %f, numerical determinant %f or %f (HSM)" %(numTrace, numDet, numDetHSM))

# Analytical trace, determinant
print("Analytical trace: %f, analytical determinant %f" %(derivedProps['trace'], derivedProps['det']))

print(np.allclose([derivedProps['trace'], derivedProps['det']], [numTrace, numDet], atol=1.e-3, rtol=1.e-3))

The numerical and analytical calculations of trace and determinant agree!

### 3.2 First moments

Of course, the second moment calculation depends on a successful calculation of the first moment, which is an input. We should've actually checked for this first.

Since HSM runs on the image, it suffers from some sub-pixel error. But we see that the numerical and analytical first moment calculations agree within the pixel precision.

In [ ]:
print(anaCentroid.x, anaCentroid.y)
print(outputHSM.moments_centroid.x, outputHSM.moments_centroid.y)

### 3.2 Ellipticities

Unfortunately, we see some differences in the leading digits between the numerical and analytical $e_1$ and $e_2$. One possible explanation is that the sub-pixel error in the HSM's first moment calculation becomes significant as we convert from pixels to arcseconds, and the error propagates into the second moment calculation. Since ellipticities are ratios between linear combinations of second moments, it is vulnerable to very small errors in either the numerator or the denominator.

In [ ]:
print("Numerical e1, e2: (%f, %f)" %(outputHSM.observed_shape.e1, outputHSM.observed_shape.e2))
print("Analytical e1, e2: (%f, %f)" %(derivedProps['e1'], derivedProps['e2']))